# PDF Beautifier Agent 사용 예제

LLM이 생성한 긴 텍스트를 예쁜 PDF로 변환하는 에이전트입니다.

## 1. 환경 설정

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from pdf_beautifier import PDFBeautifierAgent

# 환경 변수 로드
load_dotenv()

# LLM 선택 (OpenAI 또는 Anthropic)
# llm = ChatOpenAI(model="gpt-4", temperature=0.7)
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0.7)

# PDF Beautifier 에이전트 생성
agent = PDFBeautifierAgent(llm=llm)

## 2. 예제 텍스트 준비

LLM이 생성한 긴 텍스트를 시뮬레이션합니다.

In [ ]:
# 예제: LLM이 생성한 긴 텍스트
sample_text = """
인공지능의 발전과 비즈니스 혁신

최근 몇 년간 인공지능 기술은 놀라운 속도로 발전하고 있습니다. 특히 대규모 언어 모델(LLM)의 등장은 
비즈니스 환경에 혁명적인 변화를 가져오고 있습니다.

주요 변화 영역

첫째, 고객 서비스 자동화입니다. AI 챗봇과 가상 어시스턴트는 24시간 고객 지원을 제공하며, 
인간 상담원보다 빠르고 일관된 서비스를 제공합니다. 이를 통해 기업은 운영 비용을 크게 절감하면서도 
고객 만족도를 향상시킬 수 있습니다.

둘째, 데이터 분석 및 의사결정 지원입니다. AI는 방대한 양의 데이터를 실시간으로 분석하여 
비즈니스 인사이트를 제공합니다. 이는 경영진이 더 정확하고 빠른 의사결정을 내릴 수 있게 도와줍니다.

셋째, 콘텐츠 생성 및 마케팅입니다. LLM을 활용하면 마케팅 카피, 블로그 포스트, 소셜 미디어 콘텐츠 등을 
자동으로 생성할 수 있습니다. 이는 마케팅 팀의 생산성을 크게 향상시킵니다.

구현 시 고려사항

AI 도입 시 다음과 같은 사항들을 고려해야 합니다:

1. 데이터 품질: AI 모델의 성능은 학습 데이터의 품질에 크게 의존합니다.
2. 윤리적 고려사항: AI의 편향성, 개인정보 보호, 투명성 등을 고려해야 합니다.
3. 직원 교육: AI 도구를 효과적으로 활용하기 위해서는 직원 교육이 필수적입니다.
4. 단계적 도입: 한 번에 모든 것을 바꾸려 하지 말고, 파일럿 프로젝트부터 시작하는 것이 좋습니다.

결론

AI 기술은 비즈니스 환경을 근본적으로 변화시키고 있습니다. 기업들은 이러한 변화에 적극적으로 
대응하여 경쟁력을 유지하고 새로운 기회를 창출해야 합니다. 하지만 동시에 윤리적이고 책임감 있는 
AI 활용을 위해 노력해야 합니다.
"""

print("텍스트 준비 완료!")
print(f"텍스트 길이: {len(sample_text)} 글자")

## 3. PDF 생성

텍스트를 분석하고 예쁜 비즈니스 보고서 스타일의 PDF로 변환합니다.

In [ ]:
# PDF 생성
result = agent.beautify(
    text=sample_text,
    output_path="./output/business_report.pdf",
    title="AI 비즈니스 혁신 보고서",
    style="business"
)

# 결과 출력
print("\n" + "="*50)
if result["success"]:
    print(f"✅ {result['message']}")
    print(f"\n생성된 마크다운 미리보기:")
    print("-" * 50)
    print(result["markdown"][:500] + "...")
else:
    print(f"❌ {result['message']}")

## 4. HTML 미리보기 생성 (선택사항)

PDF 생성 전에 HTML로 미리보기를 확인할 수 있습니다.

In [ ]:
# HTML 미리보기 생성
preview_result = agent.preview_html(
    text=sample_text,
    output_path="./output/preview.html",
    title="AI 비즈니스 혁신 보고서",
    style="business"
)

if preview_result["success"]:
    print(f"✅ {preview_result['message']}")
    print("\nHTML 파일을 브라우저에서 열어 미리보기를 확인하세요!")

## 5. 실전 예제: LLM으로 텍스트 생성 후 PDF 변환

실제 LLM으로 텍스트를 생성하고 바로 PDF로 변환합니다.

In [ ]:
# LLM으로 긴 텍스트 생성
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 전문 비즈니스 컨설턴트입니다. 상세하고 전문적인 보고서를 작성합니다."),
    ("human", "{topic}에 대한 상세한 분석 보고서를 작성해주세요. 최소 500자 이상으로 작성하세요.")
])

chain = prompt | llm | StrOutputParser()

# 주제 선택
topic = "2024년 디지털 트랜스포메이션 트렌드"

print(f"📝 주제: {topic}")
print("LLM이 보고서를 생성하는 중...\n")

# LLM으로 텍스트 생성
generated_text = chain.invoke({"topic": topic})

print("생성된 텍스트:")
print("-" * 50)
print(generated_text[:300] + "...")
print("-" * 50)

In [ ]:
# 생성된 텍스트를 PDF로 변환
result = agent.beautify(
    text=generated_text,
    output_path="./output/digital_transformation_report.pdf",
    title=topic,
    style="business"
)

if result["success"]:
    print(f"\n✅ {result['message']}")
    print("\nPDF 파일이 생성되었습니다!")
else:
    print(f"\n❌ {result['message']}")

## 6. 마크다운을 직접 PDF로 변환

이미 구조화된 마크다운이 있다면 LLM 없이 바로 PDF로 변환할 수 있습니다.

In [ ]:
# 마크다운 예제
markdown_content = """
# 프로젝트 진행 보고서

## 개요

본 보고서는 Q4 프로젝트의 진행 상황을 요약합니다.

## 주요 성과

- **매출 증가**: 전년 대비 25% 성장
- **고객 만족도**: 4.5/5.0 달성
- **신규 고객**: 1,200명 확보

## 다음 단계

1. 마케팅 전략 개선
2. 제품 라인 확장
3. 고객 지원 강화

> **참고**: 상세한 데이터는 별도 첨부 파일 참조
"""

# 마크다운을 바로 PDF로 변환
result = agent.markdown_to_pdf(
    markdown_content=markdown_content,
    output_path="./output/project_report.pdf"
)

if result["success"]:
    print(f"✅ {result['message']}")

## 정리

### 사용 가능한 메서드

1. **`beautify()`**: 텍스트를 분석하고 PDF로 변환
   - LLM이 텍스트를 구조화
   - 자동으로 제목, 섹션 등을 생성

2. **`preview_html()`**: HTML 미리보기 생성
   - PDF 생성 전 미리보기 가능

3. **`markdown_to_pdf()`**: 마크다운을 직접 PDF로 변환
   - 이미 구조화된 마크다운이 있을 때 사용

### 스타일 옵션

- `business`: 비즈니스 보고서 스타일 (기본값)
- `academic`: 학술 논문 스타일
- `casual`: 캐주얼한 문서 스타일
